# Колокації

In [38]:
import warnings
import stanfordnlp
from typing import List, Dict

In [2]:
warnings.filterwarnings('ignore')

In [4]:
stanfordnlp.download('en')

Using the default treebank "en_ewt" for language "en".
Would you like to download the models for: en_ewt now? (Y/n)
n


In [41]:
stanfordnlp.download('uk')

Using the default treebank "uk_iu" for language "uk".
Would you like to download the models for: uk_iu now? (Y/n)
Y

Default download directory: /home/dbabenko/stanfordnlp_resources
Hit enter to continue or type an alternate directory.


Download location: /home/dbabenko/stanfordnlp_resources/uk_iu_models.zip


100%|██████████| 234M/234M [11:45<00:00, 331kB/s] 



Download complete.  Models saved to: /home/dbabenko/stanfordnlp_resources/uk_iu_models.zip
Extracting models file for: uk_iu
Cleaning up...Done.


### 1. Англійська мова

У цьому завданні вам потрібно дослідити сполучуваність дієслів одного синонімного ряду з прислівниками. Наприклад, ми частіше кажемо "love somebody dearly", "honor somebody highly", але "admire somebody greatly".

#### Завдання:

#### 1. продовжте синонімний ряд дієслів: "say", "tell", "speak", "claim", "communicate"

"say", "tell", "speak", "claim", "communicate", "express", "talk", "report", "notify", "argue", "report"

In [6]:
verbs = ["say", "tell", "speak", "claim", "communicate", 
         "express", "talk", "report", "notify", "argue", "report"]

#### 2. напишіть функцію, яка знаходить у реченні дієслово (за складеним раніше синонімним рядом) і витягає усі можливі прислівники на "-ly", якими це дієслово керує

In [7]:
nlp = stanfordnlp.Pipeline()

Use device: gpu
---
Loading: tokenize
With settings: 
{'model_path': '/home/dbabenko/stanfordnlp_resources/en_ewt_models/en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/home/dbabenko/stanfordnlp_resources/en_ewt_models/en_ewt_tagger.pt', 'pretrain_path': '/home/dbabenko/stanfordnlp_resources/en_ewt_models/en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/home/dbabenko/stanfordnlp_resources/en_ewt_models/en_ewt_lemmatizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings: 
{'model_path': '/home/dbabenko/stanfordnlp_resources/en_ewt_models/en_ewt_parser.pt', 'pretrain_path': '/home/dbabenko/stanfordnlp_resources/en_ew

##### Необхідна функція представленна нище

In [8]:
def find_verbs_with_associated_adverbs(nlp, text: str, verbs: List[str], 
                                       verb_adverb: Dict[str, Dict[str, int]] = None):
    doc = nlp(text)
    
    if verb_adverb is None:
        verb_adverb = dict()
    
    for sent in doc.sentences:
        for token in sent.dependencies:
            if token[0].lemma not in verbs:
                continue

            if token[2].upos !='ADV':
                continue


            verb = token[0].lemma
            adverb = token[2].lemma
            
            if adverb[-2:] != 'ly':
                continue

            if verb not in verb_adverb:
                verb_adverb[verb] = dict()

            if adverb not in verb_adverb[verb]:
                verb_adverb[verb][adverb] = 1
            else:
                verb_adverb[verb][adverb] += 1        
    
    return verb_adverb

##### Приклад використання даної функції

In [9]:
sentence_example = "I was curious how the Obama campaign was going to deal with this new development , since they had claimed loudly last week that al-Maliki had definitively moved away from McCain 's position and over to their own.\n"

In [10]:
find_verbs_with_associated_adverbs(nlp, text=sentence_example, verbs=verbs)

{'claim': {'loudly': 1}}

#### 3. напишіть програму, яка знайде усі можливі прислівники для ваших дієслів у корпусі блогів

In [14]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [20]:
!ls 'gdrive/My Drive/NLP/Task-1/data'

blog2008.txt


In [18]:
#corpus_file = "gdrive/My Drive/NLP/Task-1/data/blog2008.txt"
corpus_file = "../../../tasks/02-structural-linguistics/data/blog2008.txt"

In [22]:
def read_file(file: str):
    lines = []
    with open(file) as f:        
        lines = f.readlines()
            
    return lines

In [23]:
corpus = read_file(corpus_file)

In [25]:
def find_all_adverb_with_associated_verbs(nlp, corpus: List[str], verbs: List[str]):
    result = dict()
    for text in corpus:
        find_verbs_with_associated_adverbs(nlp, text=text, verbs=verbs, verb_adverb=result)
    
    return result
        

In [26]:
%time verb_assoc_adverb = find_all_adverb_with_associated_verbs(nlp, corpus[:100], verbs)

CPU times: user 4.42 s, sys: 0 ns, total: 4.42 s
Wall time: 4.42 s


In [27]:
(len(corpus) / 100) * 4.42 / 3600

3.7323707777777777

In [28]:
verb_assoc_adverb

{'claim': {'initially': 1}}

#### 4. на виході програма повинна видати десять найчастотніших прислівників для кожного дієслова (разом із частотою). Приклад виводу:


say: (loudly, 51), (silently, 45), (quietly, 10)<br/>
tell: (quietly, 100), (loudly, 61), (seriously, 5)

In [29]:
%time verb_assoc_adverb = find_all_adverb_with_associated_verbs(nlp, corpus, verbs)

CPU times: user 3h 37min 10s, sys: 35.7 s, total: 3h 37min 46s
Wall time: 3h 37min 52s


In [34]:
def print_adverb_with_associated_verbs(verb_assoc_adverb):    
    for verb in verb_assoc_adverb:
        print(f"{verb}:")
        for adverb in verb_assoc_adverb[verb]:
            count = verb_assoc_adverb[verb][adverb]
            print(f"\t({adverb}, {count})")
            
        print('')

#### 5. збережіть програму та результати аналізу на корпусі (вивід) у директорії з вашим іменем


In [35]:
print_adverb_with_associated_verbs(verb_assoc_adverb)

claim:
	(initially, 3)
	(brazenly, 1)
	(surely, 2)
	(solemnly, 1)
	(previously, 8)
	(falsely, 78)
	(oddly, 1)
	(apparently, 2)
	(only, 6)
	(fiercely, 1)
	(alternatively, 1)
	(constantly, 1)
	(loudly, 1)
	(annually, 1)
	(bizarrely, 2)
	(actually, 3)
	(piously, 1)
	(really, 2)
	(recently, 4)
	(preposterously, 1)
	(notably, 1)
	(widely, 1)
	(hardly, 1)
	(absurdly, 3)
	(clearly, 2)
	(probably, 1)
	(finally, 1)
	(namely, 1)
	(illegally, 1)
	(righteously, 1)
	(indignantly, 2)
	(immediately, 2)
	(explicitly, 1)
	(tearfully, 1)
	(patronizingly, 1)
	(credibly, 3)
	(repeatedly, 8)
	(openly, 1)
	(remotely, 1)
	(erroneously, 2)
	(publicly, 3)
	(consistently, 3)
	(realistically, 1)
	(truly, 1)
	(completely, 1)
	(originally, 2)
	(merely, 1)
	(chronologically, 1)
	(incorrectly, 2)
	(evidently, 1)
	(inadvertently, 1)
	(persuasively, 1)
	(predictably, 1)
	(likely, 1)
	(ostensibly, 1)
	(laughably, 3)
	(sanctimoniously, 1)
	(shamelessly, 1)
	(reportedly, 1)
	(ironically, 2)
	(cluelessly, 1)
	(angrily, 1)

### 2. Українська мова

In [42]:
nlp_uk = stanfordnlp.Pipeline(lang="uk")

Use device: gpu
---
Loading: tokenize
With settings: 
{'model_path': '/home/dbabenko/stanfordnlp_resources/uk_iu_models/uk_iu_tokenizer.pt', 'lang': 'uk', 'shorthand': 'uk_iu', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/home/dbabenko/stanfordnlp_resources/uk_iu_models/uk_iu_tagger.pt', 'pretrain_path': '/home/dbabenko/stanfordnlp_resources/uk_iu_models/uk_iu.pretrain.pt', 'lang': 'uk', 'shorthand': 'uk_iu', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/home/dbabenko/stanfordnlp_resources/uk_iu_models/uk_iu_lemmatizer.pt', 'lang': 'uk', 'shorthand': 'uk_iu', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings: 
{'model_path': '/home/dbabenko/stanfordnlp_resources/uk_iu_models/uk_iu_parser.pt', 'pretrain_path': '/home/dbabenko/stanfordnlp_resources/uk_iu_models/uk_iu

#### 1. напишіть функцію, яка знаходить у реченні іменник, що позначає істоту, і витягає усі прикметники, якими цей іменник керує


In [43]:
def is_token_anim(token):
    if 'Animacy=Anim' in token.feats:
        return True
    return False

In [73]:
def is_noun_adj_connection(token):
    if token[0].upos == 'ADJ':
        if token[2].upos == 'NOUN' or token[2].upos == 'PROPN':
            return True
        
    if token[0].upos == 'NOUN' or token[0].upos == 'PROPN':
        if token[2].upos == 'ADJ':
            return True
        
    return False


In [144]:
def find_noun_with_associated_adj(nlp, text: str, noun_adj_dict : Dict[str, Dict[str, int]] = None):
    doc = nlp(text)
    
    if noun_adj_dict is None:
        noun_adj_dict = dict()
    
    for sent in doc.sentences:
        for token in sent.dependencies:   
#             print(token)
            
            if is_noun_adj_connection(token) is False:
                continue
                     
            noun_token = token[0] if token[0].upos != 'ADJ' else token[2]
            
            #перевірка чи іменнки позначає істоту
#             print(noun_token)
            if is_token_anim(noun_token) is False:
                continue
            
            adj_token = token[0] if token[0].upos == 'ADJ' else token[2]
            
            noun = noun_token.text.lower()
            adjective = adj_token.text.lower()
            
            if noun not in noun_adj_dict:
                noun_adj_dict[noun] = { adjective : 1}
            else:
                if adjective not in noun_adj_dict[noun]:
                    noun_adj_dict[noun][adjective] = 1
                else:
                    noun_adj_dict[noun][adjective] += 1
                            
    return noun_adj_dict        

In [145]:
sentence_example = "Чоловік був щирий та добре поводився зі своїм котом."

In [146]:
find_noun_with_associated_adj(nlp_uk, sentence_example)

{'чоловік': {'щирий': 1}}

#### 2. напишіть програму, яка знайде усі можливі словосполучення прикметників та іменників-істот у романі "Тигролови"

In [147]:
tyhrolovy_file = "../../../tasks/02-structural-linguistics/data/tyhrolovy.txt"

In [148]:
tyhrolovy_corpus = read_file(tyhrolovy_file)

In [149]:
def find_noun_with_associated_adj_for_corpus(nlp, corpus: List[str]):
    result = dict()
    for text in corpus:
        if text is None or len(text) <= 1:
            continue 
        find_noun_with_associated_adj(nlp, text, result)
    return result

In [150]:
find_noun_with_associated_adj(nlp_uk, text)

{'сірко': {'старий': 1}}

In [151]:
noun_adj_dict = find_noun_with_associated_adj_for_corpus(nlp_uk, tyhrolovy_corpus[:50])

In [152]:
noun_adj_dict

{'лось': {'сохатий': 1},
 'дракон': {'легендарний': 1, 'реальний': 2, 'справжній': 1, 'найбільший': 1},
 'драконів': {'найстрашніший': 1},
 'голова': {'вогненноока': 1},
 'циклоп': {'велетенський': 1, 'двоокий': 1},
 'мертвяків': {'живих': 1},
 'дракона': {'натоптане': 1},
 'й': {'циклопічний': 1}}

#### 3. на виході програма повинна видати усі знайдені словосполучення та їх частоти. Приклад виводу: <br/> 
84: старий сірко <br/> 
5: великий начальник <br/> 

In [153]:
%time noun_adj_dict = find_noun_with_associated_adj_for_corpus(nlp_uk, tyhrolovy_corpus)

CPU times: user 3min 45s, sys: 724 ms, total: 3min 46s
Wall time: 3min 46s


In [162]:
def get_phrases(noun_adj_dict):
    phrase_count = list()
    for noun in noun_adj_dict:
        for adjective in noun_adj_dict[noun]:
            phrase = f"{adjective} {noun}"
            count = noun_adj_dict[noun][adjective]
            phrase_count.append((phrase, count))
    phrase_count.sort(key=lambda r:r[1], reverse=True)
    return phrase_count


In [163]:
phrase_count_list = get_phrases(noun_adj_dict)

In [165]:
def print_phrase_count_list(phrase_count_list):    
    for item in phrase_count_list:
        print(item[1], ": ", item[0])

#### 4. збережіть програму та результати виводу в директорії з вашим іменем

In [167]:
print_phrase_count_list(phrase_count_list)

81 :  старий сірко
7 :  старий мороз
6 :  стара сірчиха
5 :  цибатий зять
4 :  сонячні зайчики
3 :  смугастий звірок
3 :  якутський пес
3 :  великий пес
3 :  старого сірка
3 :  дурний дід
3 :  старим сірком
2 :  реальний дракон
2 :  великий начальник
2 :  рідний товаришу
2 :  останні пак
2 :  зв'язана кішка
2 :  молодий олень
2 :  бронзових якутів
2 :  владивостоцький турок
2 :  тую рись
2 :  хутряного звіра
2 :  волохатий бог
2 :  могутній бог
2 :  сідлані коні
2 :  вірний пес
2 :  вража дівчина
2 :  добрі люди
2 :  справжній мисливець
2 :  дика коза
2 :  дикі кози
2 :  насуплена наталка
2 :  вражої мами
2 :  нав'ючених коней
2 :  плямистого оленя
2 :  горда тварина
2 :  приголомшений григорій
2 :  старі друзі
2 :  старого мороза
2 :  гарна дочка
2 :  живого тигра
1 :  сохатий лось
1 :  легендарний дракон
1 :  справжній дракон
1 :  найбільший дракон
1 :  найстрашніший драконів
1 :  вогненноока голова
1 :  примружені голова
1 :  непокрита голова
1 :  оленяча голова
1 :  мокра голова
1 